# NOTEBOOK PARA OBTENER LOS HITS Y MEJORES HITS
Esto incluye la limpieza que se requiere ya que se tienen dobles espacios en cada línea.

**Que pasos debo seguir para elegir todos los raw hits de acuerdo a lo indicado por Salva:**

"...En realidad, hablando con mi hermano hemos decidido que los selected pulses no cal guardarlos, los usas para el match, y coges todos los hits que coincidan (cada uno de ellos será una nueva fila en tu CVS). Como decía antes, en las columnas tendrás toda la información del muón, del bbfit, del affit, y del hit coincidente con un selected pulse particular."

El pseudocódigo que se me ocurre es el siguiente:

1. Seleccionar el primer "selected pulse" de la lista de seleccionados.
2. Filtrar el df de raw hits por todos los que coincidan en linea detector y piso detector.
3. Filtrar el resultado viendo cuales coinciden en los parámetrox X', Y', Z' y X, Y, Z.
4. Guardar el resultado en el dataframe final junto con runID, frameID
5. Iterar hasta que ya no haya mas selected pulses
6. Hacer lo mismo para cada evento del archivo seleccionado

Actualización: Salva me mencionó en correo del día 3 Jul que se debe filtrar con respecto al piso_detector y línea_detector y quedarse con todos esos hits. Cito a la letra: 

"Perdón por el mareo. A ver, hablando con mi hermano me ha recordado que la forma que quedamos de hacerlo es con match de linea y piso del sensor, mas un cutoff de tiempo relativo respecto al selected pulse, que dijimos por ejemplo 50.
Yo recomendaria este filtro ponerlo mas adelante y coger todos los raw hits qcoinciden en linia y piso, mas qnada porque ai queremos probar otro cutoff de 70 o 100, no tengamos que correr todo esto de nuevo..."

Con base en esta nueva instrucción intentaré seleccionar de esta forma los best hits. (Sólo con linea y piso).

Referencias:
* https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/

* https://medium.com/@juan.7655/cleansing-messy-excel-files-with-pandas-95a2ba3d8b2d

* https://medium.com/dunder-data/pandas-challenge-2-explain-the-1-000x-speed-difference-when-taking-the-mean-35d72fe10c1

In [1]:
#### Primero algunas variables importantes de ubicación de los archivos y librerías ####

import os
import pandas as pd
import re
import math

#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi PC Desktop

extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
csv_path = "F:\\DATA_TFM\\CSVs\\" #Path para mi PC en el trabajo

#extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
#csv_path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi latptop en casa

txtfilesarr = os.listdir(extractpath); txtfilesarr = sorted(txtfilesarr)
csvarr = os.listdir(csv_path); csvarr = sorted(csvarr)

In [2]:
###############################
#### FUNCIONES IMPORTANTES ####
###############################

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para limpiar un hit (lista) de elementos tipo dobles espacios y simples
def limpiahit(templist):
    cleanlist = []
    for j in range(len(templist)):
        if templist[j]!="":
            cleanlist.append(templist[j])
    return(cleanlist)

#Función para buscar los "rows" que coinciden en línea y piso de solo un selected pulse
def func_lineas_coinc(linea,piso,selp,hits):
    lineas_coinc = []
    for i in range(len(hits)):
        if hits[i][2]==linea and hits[i][3]==piso:
            lineas_coinc.append(hits_raw_limpios[i])
    return(lineas_coinc)

#Función para obtener todas las líneas de los selected pulse de un evento
def func_lineas_evento(sel_hits_limpios,hits_raw_limpios):
    lineas_selp_evento = []
    for i in range(len(sel_hits_limpios)):
        lindet = sel_hits_limpios[i][2] #Linea detector a buscar
        storey = sel_hits_limpios[i][3] #Piso detector a buscar
        lineas = func_lineas_coinc(lindet,storey,sel_hits_limpios[i],hits_raw_limpios)
        #Debo añadir las lineas de cada selected pulse a otra lista como "renglones"
        for j in range(len(lineas)):
            lineas_selp_evento.append(lineas[j])
    return(lineas_selp_evento)

## Como ya he visto que trabajar con listas es muchísimo más rapido intentaré hacerlo sólo de esta manera (con listas), al final sí generaré un dataframe, pero sólo hasta el final.
    
## Primero debemos tomar el primer selected pulse como pivote y después debemos comparar contra todas las líneas que coincidan en linea detector y piso detector y seleccionarlas.

In [ ]:
###################################
### OBTENIENDO LOS MEJORES HITS ###
###################################

###### CÓDIGO PARA RECORRER TODOS LOS ARCHIVOS ######

#### FILTRO PARA BUSCAR LOS HITS QUE HACEN MATCH CON EL SELECTED PULSE Y QUEDARNOS 
#### CON LA DATA DE MODULO OPTICO (ENTIENDASE RENGLON O RENGLONES QUE CORRESPONDEN AL SELECTED HIT)
#### EL CRITERIO ES QUEDARSE CON TODOS LOS RAW HITS QUE SEAN IGUALES EN X',Y',Z',X,Y, y Z.

inicio = 432   #En este caso como trabaje los archivos con otra PC, debo empezar en este índice.
best_hits = [] #Definimos la lista donde se guardarán los hits seleccionados

#Primer bucle donde se leerá la data de cada archivo
for z in range(1406):
    #Abrimos el archivo, de acuerdo con el índice z
    file = csv_path+csvarr[inicio]
    data = pd.read_csv(file, header=None)
    #Obteniendo los indices necesarios
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)
    print("El archivo trabajando es:",file,"eventos:",len(ind_startev))
    #Obtenemos la metadata runID e interactionID ya que es la misma para cada archivo
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0]) #Corregí esta línea era 0 en ambos
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    print("Entrando al bucle para obtener los best-hits")
    bhcount_file = 0; rhcount_file = 0 #Contadores para "monitorear"
    ### Bucle para buscar los best hits de cada archivo en cada evento del archivo
    for j in range(len(ind_startev)):
        #Obtengo el frameID y trigger_counter, que son diferentes para cada evento
        temprunid = list(data[0][ind_runid[j]].split(" "))
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Encontramos primero el # de lineas de rawhits y selected pulses
        difrawhits = ind_bb_selp[j] - ind_bbfit[j] #debe variar con la variable del bucle de eventos
        difselhits = ind_endev[j] - ind_bb_selp[j] #debe variar con la variable del bucle de eventos
        start_rawhit = ind_bbfit[j]+1; #debe variar con el bucle de eventos
        start_selhit = ind_bb_selp[j]+1 #debe variar con el bucle de eventos
        raw_hits = []; hits_raw_limpios = [];sel_hits = []; sel_hits_limpios = []
        #Obteniendo solo los raw_hits de un evento
        for i in range(difrawhits-1):
            temprawhit = list(data[0][start_rawhit+i].split(" "))
            raw_hits.append(temprawhit)
            #Limpiamos los raw hits obtenidos del evento
        for i in range(len(raw_hits)): hits_raw_limpios.append(limpiahit(raw_hits[i]))
        #Obteniendo solo los selected pulses (hits) del mismo evento
        for i in range(difselhits-1):
            tempselhit = list(data[0][start_selhit+i].split(" "))
            sel_hits.append(tempselhit)
        #Limpiamos los raw hits obtenidos del evento
        for i in range(len(sel_hits)): sel_hits_limpios.append(limpiahit(sel_hits[i]))
        #A continuación obtenemos todas las lineas que coinciden con cada selected pulse del evento
        templist = func_lineas_evento(sel_hits_limpios,hits_raw_limpios)
        #Agregamos la metadata que le falta a cada "row"
        for i in range(len(templist)):
            templist[i].insert(0,interactionID)
            templist[i].insert(0,trigger_counter)
            templist[i].insert(0,frameID)
            templist[i].insert(0,runID)
        for k in range(len(templist)):
            best_hits.append(templist[k])
        rhcount_file += len(raw_hits)
        bhcount_file += len(templist)
    print("Raw_hits:",rhcount_file,"Número de best_hits:",bhcount_file)
    inicio += 1 #Incremento para abrir el archivo correcto

print("\nFin de la extracción de la data")

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058730_numu_CC_a_reco.i3.csv eventos: 1396
Entrando al bucle para obtener los best-hits
Raw_hits: 408751 Número de best_hits: 39685
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058770_anumu_CC_a_reco.i3.csv eventos: 597
Entrando al bucle para obtener los best-hits
Raw_hits: 287016 Número de best_hits: 26866
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058770_numu_CC_a_reco.i3.csv eventos: 578
Entrando al bucle para obtener los best-hits
Raw_hits: 278775 Número de best_hits: 26565
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058790_anumu_CC_a_reco.i3.csv eventos: 90
Entrando al bucle para obtener los best-hits
Raw_hits: 37691 Número de best_hits: 2295
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058790_numu_CC_a_reco.i3.csv eventos: 665
Entrando al bucle para obtener los best-hits
Raw_hits: 298915 Número de best_hits: 27807
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_058810_anumu_CC_a_reco.i3.csv eventos: 1348
Entrando al bucle para ob

Raw_hits: 358131 Número de best_hits: 37536
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059870_anumu_CC_a_reco.i3.csv eventos: 1296
Entrando al bucle para obtener los best-hits
Raw_hits: 371607 Número de best_hits: 35612
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059870_numu_CC_a_reco.i3.csv eventos: 1199
Entrando al bucle para obtener los best-hits
Raw_hits: 344214 Número de best_hits: 34336
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059880_anumu_CC_a_reco.i3.csv eventos: 1389
Entrando al bucle para obtener los best-hits
Raw_hits: 436311 Número de best_hits: 38283
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059880_numu_CC_a_reco.i3.csv eventos: 1336
Entrando al bucle para obtener los best-hits
Raw_hits: 419884 Número de best_hits: 38524
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059890_numu_CC_a_reco.i3.csv eventos: 1213
Entrando al bucle para obtener los best-hits
Raw_hits: 396229 Número de best_hits: 37311
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_059900_anumu_CC_a_

Raw_hits: 385379 Número de best_hits: 36721
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060930_numu_CC_a_reco.i3.csv eventos: 1243
Entrando al bucle para obtener los best-hits
Raw_hits: 353860 Número de best_hits: 33954
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060940_anumu_CC_a_reco.i3.csv eventos: 1643
Entrando al bucle para obtener los best-hits
Raw_hits: 475062 Número de best_hits: 42675
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060940_numu_CC_a_reco.i3.csv eventos: 1490
Entrando al bucle para obtener los best-hits
Raw_hits: 428839 Número de best_hits: 37734
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060970_anumu_CC_a_reco.i3.csv eventos: 1690
Entrando al bucle para obtener los best-hits
Raw_hits: 443335 Número de best_hits: 41540
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060970_numu_CC_a_reco.i3.csv eventos: 1550
Entrando al bucle para obtener los best-hits
Raw_hits: 405964 Número de best_hits: 38241
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_060990_anumu_CC_a_

Raw_hits: 418536 Número de best_hits: 39931
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061470_numu_CC_a_reco.i3.csv eventos: 1475
Entrando al bucle para obtener los best-hits
Raw_hits: 389404 Número de best_hits: 38106
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061480_anumu_CC_a_reco.i3.csv eventos: 1603
Entrando al bucle para obtener los best-hits
Raw_hits: 401059 Número de best_hits: 38912
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061480_numu_CC_a_reco.i3.csv eventos: 1490
Entrando al bucle para obtener los best-hits
Raw_hits: 373773 Número de best_hits: 37212
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061730_anumu_CC_a_reco.i3.csv eventos: 1237
Entrando al bucle para obtener los best-hits
Raw_hits: 381898 Número de best_hits: 37886
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061730_numu_CC_a_reco.i3.csv eventos: 1138
Entrando al bucle para obtener los best-hits
Raw_hits: 349738 Número de best_hits: 34156
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_061740_anumu_CC_a_

Raw_hits: 337533 Número de best_hits: 33825
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062440_anumu_CC_a_reco.i3.csv eventos: 1046
Entrando al bucle para obtener los best-hits
Raw_hits: 357395 Número de best_hits: 30763
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062440_numu_CC_a_reco.i3.csv eventos: 931
Entrando al bucle para obtener los best-hits
Raw_hits: 322316 Número de best_hits: 29979
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062450_anumu_CC_a_reco.i3.csv eventos: 1287
Entrando al bucle para obtener los best-hits
Raw_hits: 366239 Número de best_hits: 37648
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062450_numu_CC_a_reco.i3.csv eventos: 1214
Entrando al bucle para obtener los best-hits
Raw_hits: 342656 Número de best_hits: 33616
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062460_anumu_CC_a_reco.i3.csv eventos: 1324
Entrando al bucle para obtener los best-hits
Raw_hits: 352898 Número de best_hits: 36790
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_062460_numu_CC_a_r

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064600_numu_CC_a_reco.i3.csv eventos: 837
Entrando al bucle para obtener los best-hits
Raw_hits: 369321 Número de best_hits: 27461
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064630_anumu_CC_a_reco.i3.csv eventos: 308
Entrando al bucle para obtener los best-hits
Raw_hits: 161940 Número de best_hits: 12367
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064630_numu_CC_a_reco.i3.csv eventos: 245
Entrando al bucle para obtener los best-hits
Raw_hits: 124850 Número de best_hits: 8335
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064640_anumu_CC_a_reco.i3.csv eventos: 339
Entrando al bucle para obtener los best-hits
Raw_hits: 197565 Número de best_hits: 13930
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064640_numu_CC_a_reco.i3.csv eventos: 349
Entrando al bucle para obtener los best-hits
Raw_hits: 202949 Número de best_hits: 14409
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_064680_anumu_CC_a_reco.i3.csv eventos: 26
Entrando al bucle para obt

Raw_hits: 293378 Número de best_hits: 22152
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065410_numu_CC_a_reco.i3.csv eventos: 773
Entrando al bucle para obtener los best-hits
Raw_hits: 314625 Número de best_hits: 25112
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065440_anumu_CC_a_reco.i3.csv eventos: 1015
Entrando al bucle para obtener los best-hits
Raw_hits: 394477 Número de best_hits: 31355
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065440_numu_CC_a_reco.i3.csv eventos: 960
Entrando al bucle para obtener los best-hits
Raw_hits: 372013 Número de best_hits: 30573
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065480_anumu_CC_a_reco.i3.csv eventos: 664
Entrando al bucle para obtener los best-hits
Raw_hits: 343566 Número de best_hits: 27594
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065480_numu_CC_a_reco.i3.csv eventos: 618
Entrando al bucle para obtener los best-hits
Raw_hits: 320339 Número de best_hits: 26892
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_065500_anumu_CC_a_reco

In [ ]:
#Pasamos a dataframe la lista con todos los hits seleccionados, en este caso de un archivo
print(len(best_hits))
columnas = ['runID','frameID','trigger_counter','interactionID','hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
df = pd.DataFrame(best_hits,columns=columnas)

In [ ]:
df.sample(5)

In [ ]:
#Guardamos el dataframe como un archivo CSV para unirlo con el otro (Laptop casa) ya procesado
#Todavía falta generar la transformación azimut y zenit pero eso será cuando ya tenga el dataframe completo

#outputpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
outputpath = "F:\DATA_TFM\CLEANDATA\\"
    
df.to_csv(outputpath+"data_best_hits.csv", index = None)#, header=None)

## A PARTIR DE AQUI FUE LO PRIMERO QUE REALIZE PARA LLEGAR AL ALGORITMO Y PROGRAMA DE LAS PRIMERAS CELDAS.

In [ ]:
###################################
### OBTENIENDO LOS MEJORES HITS ###
###################################

## Por el momento será para un solo archivo, posteriormente haré el bucle.

#Leyendo el archivo csv ya sin las líneas no deseadas en un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

#Obteniendo los indices necesarios
ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

In [ ]:
#Ya que tenemos dos funciones, 1 que nos da las lineas que coinciden con el selected puls por linea y piso
#y otra que nos da todas las lineas que coinciden con todos los selp de un evento
#hay que proceder a iterar para todos los eventos de un mismo archivo

best_hits = []

#Requerimos el runID, el cual es el mismo para todos los selected pulses de todos los eventos en un mismo archivo
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
#Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()

#Requerimos tambien el frameID, este dato si cambia por cada evento
#Por ultimo necesitamos el trigger_counter, que cambia por cada evento

for j in range(len(ind_startev)):
    #Obtengo el frameID y trigger_counter, que son diferentes para cada evento
    temprunid = list(data[0][ind_runid[j]].split(" "))
    frameID = temprunid[1]; trigger_counter = temprunid [2]
    #Encontramos primero el # de lineas de rawhits y selected pulses
    difrawhits = ind_bb_selp[j] - ind_bbfit[j] #debe variar con la variable del bucle de eventos
    difselhits = ind_endev[j] - ind_bb_selp[j] #debe variar con la variable del bucle de eventos
    start_rawhit = ind_bbfit[j]+1; #debe variar con el bucle de eventos
    start_selhit = ind_bb_selp[j]+1 #debe variar con el bucle de eventos
    raw_hits = []; hits_raw_limpios = [];sel_hits = []; sel_hits_limpios = []
    #Obteniendo solo los raw_hits de un evento
    for i in range(difrawhits-1):
        temprawhit = list(data[0][start_rawhit+i].split(" "))
        raw_hits.append(temprawhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(raw_hits)): hits_raw_limpios.append(limpiahit(raw_hits[i]))
    #Obteniendo solo los selected pulses (hits) del mismo evento
    for i in range(difselhits-1):
        tempselhit = list(data[0][start_selhit+i].split(" "))
        sel_hits.append(tempselhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(sel_hits)): sel_hits_limpios.append(limpiahit(sel_hits[i]))
    #A continuación obtenemos todas las lineas que coinciden con cada selected pulse del evento
    templist = func_lineas_evento(sel_hits_limpios,hits_raw_limpios)
    #Agregamos la metadata que le falta a cada "row"
    for i in range(len(templist)):
        templist[i].insert(0,interactionID)
        templist[i].insert(0,trigger_counter)
        templist[i].insert(0,frameID)
        templist[i].insert(0,runID)
    for k in range(len(templist)):
        best_hits.append(templist[k])

In [ ]:
#Pasamos a dataframe la lista con todos los hits seleccionados, en este caso de un archivo
print(len(best_hits))
columnas = ['runID','frameID','trigger_counter','interactionID','hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
df = pd.DataFrame(best_hits,columns=columnas)

In [ ]:
tempdf = df.loc[df['runID'].isin(["25800"])]
tempdf.tail(5)

In [ ]:
print("El tamaño del dataframe es:",df.shape)

tempdf = df.loc[df['runID'].isin(["25800"])]

#tempdf.loc[tempdf['mód_opt'].isin(["1"])]

In [ ]:
df.sample(10)

In [ ]:
#Función que hice para buscar un frameID en específico en el dataframe final
def func_frameID(num,df):
    tempdf = df.loc[df['frameID'].isin([str(num)])]
    #print(tempdf.head(5))
    return(tempdf)

In [ ]:
otrodf = func_frameID(45225,df)

In [ ]:
otrodf

In [ ]:
#### FILTRO PARA BUSCAR LOS HITS QUE HACEN MATCH CON EL SELECTED PULSE Y QUEDARNOS 
#### CON LA DATA DE MODULO OPTICO (ENTIENDASE RENGLON O RENGLONES QUE CORRESPONDEN AL SELECTED HIT)
#### EL CRITERIO ES QUEDARSE CON TODOS LOS RAW HITS QUE SEAN IGUALES EN X',Y',Z',X,Y, y Z.

#Abro el archivo a trabajar y paso el contenido a un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

#Obteniendo los indices necesarios
#ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

#Variables auxiliares
columnas = ['hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
df_hits = pd.DataFrame(columns=columnas)
#Obtengo el runID que es el mismo para cada archivo
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
#Obtengo el frameID que es diferente para cada evento
temprunid = list(data[0][ind_runid[1]].split(" ")) #Esto va en el bucle para recorrer todos los archivos
frameID = temprunid[1]; trigger_counter = temprunid [2]
print("runID:",runID,"frameID:",frameID)

#Este bucle recorre cada row de los selected pulses
for j in range(sel_hits_ev_df.shape[0]):
    selp = sel_hits_ev_df.iloc[j][5:11]
    #Este bucle recorre cada row de los raw hits seleccionados.
    lindet_search = sel_hits_ev_df.iloc[j][2] #Linea detector a buscar
    storey_search = sel_hits_ev_df.iloc[j][3] #Piso detector a buscar
    #Primero busqué por piso detector (me pareció que era más corto ya que son menos lineas)
    tempdf = raw_hits_ev_df.loc[raw_hits_ev_df['piso_det'].isin([storey_search])]
    #Posteriormente filtré por línea del detector
    tempdf = tempdf.loc[tempdf['lin_det'].isin([lindet_search])]
    #print(tempdf)
    for i in range(tempdf.shape[0]):
        temp = tempdf.iloc[i][5:11]
        if selp.equals(temp):
            df_hits=df_hits.append(tempdf.iloc[i])
        #print("\n")